In [1]:
import sys

sys.path.append("../../")

from models.Majority_vote import MV
from stats import evaluate_perf_utils as eval_perf
import utils.load_csv as load_csv
import plotly.graph_objects as go
import numpy as np

2024-12-12 10:41:13.457341: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733996473.475597   44344 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733996473.481018   44344 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-12 10:41:13.499755: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/thibaud/majority-vote-outlier-detection/.venv/lib/python3.12/site-packages/torchvision/transforms/v2/_deprecated.py:42:

In [2]:
mv = MV.Majority_Vote(som_model_path="../../models/SOM/hpc_SOM_model.pkl", vae_model_path="../../models/VAE/vae.pth", cnn_model_path="../../models/CNN/cnn.keras") 
all_data = load_csv.load_pandas()
_, val_set, test_set = load_csv.split_data(all_data)
mv.set_thresholds(val_set)
predictions = mv.predict(test_set)

performance = eval_perf.evaluate_performance(mv.cnn.y_test, predictions)

tn = performance["tn"]
fp = performance["fp"]
fn = performance["fn"]
tp = performance["tp"]

# Confusion matrix
cm = np.array([[tn, fp], [fn, tp]])

total = cm.sum()
cm_percentage = cm / total * 100

# Create a heatmap for the confusion matrix with percentages
fig = go.Figure(
    data=go.Heatmap(
        z=cm_percentage,
        x=["Pred: 0", "Pred: 1"],
        y=["True: 0", "True: 1"],
        colorscale="Peach",  # A more pleasant color palette
        colorbar=dict(title="Percentage", tickvals=[0, 100]),
        showscale=True,
    )
)

# Add titles, labels, and precision, recall, F1 score as annotation
fig.update_layout(
    title="Confusion Matrix (Percentage)",
    xaxis_title="Predicted Labels",
    yaxis_title="True Labels",
    width=500,  # Set the width of the plot
    height=400,  # Set the height of the plot
)

# Add text annotations to show percentage values in each cell
for i in range(cm_percentage.shape[0]):
    for j in range(cm_percentage.shape[1]):
        fig.add_annotation(
            x=j,
            y=i,
            text=f"{cm_percentage[i, j]:.2f}%",
            showarrow=False,
            font=dict(size=14, color="black"),
            align="center",
        )

# Show the plot
fig.show()

Model loaded from ../../models/SOM/hpc_SOM_model.pkl.


/home/thibaud/majority-vote-outlier-detection/models/Majority_vote/../../utils/load_save_models.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(to

Threshold set at 2.5560 (Percentile: 90)
Existing model found. Only loading test data.


/home/thibaud/majority-vote-outlier-detection/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step
Test Accuracy: 95.093
Shape of SOM output: (2914,)
Shape of VAE output: (2914,)
Shape of CNN output: (2914,)
Test set shape: (2914, 7)
